In [14]:
import os
from azure.identity import DefaultAzureCredential,get_bearer_token_provider
from langchain_openai import AzureChatOpenAI
from langchain.agents import tool
from langgraph.graph import Graph
from IPython.display import display,Image
import tiktoken

# Use OpenAI-compatible encoding (gpt-4 or gpt-4o)
encoding = tiktoken.encoding_for_model("gpt-4")


token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
model= AzureChatOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint="https://azopenai-langchain.openai.azure.com/",
    azure_ad_token_provider= token_provider,
    azure_deployment= "gpt-4o-mini",
    model= "gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

def llm_response(input:str) -> str:
    response=model.invoke(input)
    print(response.content)
    return response.content

def token_count(response_text: str) -> dict:
    tokens = encoding.encode(response_text)
    return {**response_text, "token_count": len(tokens)}

graph= Graph()
graph.add_node(node ="llm_response",action=llm_response)
graph.add_node(node="token_count",action=token_count)
graph.set_entry_point("llm_response")
graph.set_finish_point("token_count")
graph.add_edge("llm_response","token_count")

compiledGraph = graph.compile()
#display(Image(data=compiledGraph.get_graph().draw_mermaid_png()))
compiledGraph.invoke("What is the capital of Japan?")

The capital of Japan is Tokyo.


TypeError: 'str' object is not a mapping